In [1]:
import findspark
findspark.init('/home/mysparkub/spark-3.0.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [7]:
from pyspark.sql.functions import countDistinct, avg, stddev, format_number
from pyspark.sql.functions import mean
from pyspark.sql.functions import max as pys_max
from pyspark.sql.functions import (dayofmonth, dayofweek, dayofyear,
                                  month, year, weekofyear,hour,
                                  format_number, date_format)

In [4]:
spark = SparkSession.builder.appName('basics').getOrCreate()

In [6]:
df = spark.read.json('files/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json')

In [7]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [8]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [9]:
df.columns

['age', 'name']

In [11]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [12]:
from pyspark.sql.types import StructField, StringType,IntegerType,StructType

In [13]:
data_schema = [StructField('age',IntegerType(), True),
              StructField('name', StringType(), True )]

In [15]:
final_struc = StructType(fields = data_schema)

In [16]:
df = spark.read.json('files/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json', schema=final_struc)

In [18]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [19]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [24]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [33]:
df.head(2)[0]

Row(age=None, name='Michael')

In [34]:
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [38]:
df.withColumn('newage', df['age']*2).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    60|
|  19| Justin|    38|
+----+-------+------+



In [40]:
df.withColumnRenamed('age', 'age2').show()

+----+-------+
|age2|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [41]:
df.createOrReplaceTempView('people')

In [42]:
results = spark.sql("select * from people")

In [43]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [44]:
results2 = spark.sql("select * from people where age>20")

In [45]:
results2.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [46]:
df2 = spark.read.csv('files/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/appl_stock.csv', inferSchema=True, header=True)

In [47]:
df2.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [52]:
df2.head(2)[0]

Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)

In [51]:
df2.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [56]:
# df2.filter("Close< 500").select('Open').show()

In [60]:
df2.filter(df2['Close'] < 500).select('Volume').show()

+---------+
|   Volume|
+---------+
|123432400|
|150476200|
|138040000|
|119282800|
|111902700|
|115557400|
|148614900|
|151473000|
|108223500|
|148516900|
|182501900|
|153038200|
|152038600|
|220441900|
|266424900|
|466777500|
|430642100|
|293375600|
|311488100|
|187469100|
+---------+
only showing top 20 rows



In [62]:
# df2.filter((df2['Close']<200) & (df2['Open']>200)).show()

In [64]:
df2.filter(df2['Low'] == 197.16).show()

+----------+------------------+----------+------+------+---------+---------+
|      Date|              Open|      High|   Low| Close|   Volume|Adj Close|
+----------+------------------+----------+------+------+---------+---------+
|2010-01-22|206.78000600000001|207.499996|197.16|197.75|220441900|25.620401|
+----------+------------------+----------+------+------+---------+---------+



In [65]:
result = df2.filter(df2['Low'] == 197.16).collect()

In [66]:
result

[Row(Date='2010-01-22', Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]

In [67]:
row = result[0]
row.asDict()

{'Date': '2010-01-22',
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

In [68]:
df= spark.read.csv('files/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/sales_info.csv', inferSchema=True, header=True)

In [69]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [70]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [76]:
df.groupBy('Company').max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [78]:
df.groupby('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [79]:
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [80]:
group_data = df.groupby('Company')

In [82]:
group_data.agg({'Sales':'sum'}).show()

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



In [87]:
from pyspark.sql.functions import countDistinct, avg, stddev, format_number
from pyspark.sql.functions import mean
from pyspark.sql.functions import max as pys_max
from pyspark.sql.functions import (dayofmonth, dayofweek, dayofyear,
                                  month, year, weekofyear,hour,
                                  format_number, date_format)

In [85]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [86]:
df.select(avg('Sales').alias('Average')).show()

+-----------------+
|          Average|
+-----------------+
|360.5833333333333|
+-----------------+



In [89]:
df.select(stddev('Sales').alias('std')).show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [91]:
df.select(stddev('Sales').alias('std')).select(format_number('std', 2).alias('Standard dv')).show()

+-----------+
|Standard dv|
+-----------+
|     250.09|
+-----------+



In [92]:
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [93]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [94]:
df= spark.read.csv('files/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/ContainsNull.csv', inferSchema=True, header=True)

In [95]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [96]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [99]:
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [100]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [101]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [102]:
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [103]:
df.na.fill('Fill Value').show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|Fill Value| null|
|emp3|Fill Value|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [104]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [105]:
df.na.fill('Fill Value',subset=['Name']).show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|Fill Value| null|
|emp3|Fill Value|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [163]:
from pyspark.sql.functions import mean
from pyspark.sql.functions import max as pys_max
from pyspark.sql.functions import (dayofmonth, dayofweek, dayofyear,
                                  month, year, weekofyear,hour,
                                  format_number, date_format)

In [110]:
mean_val = df.select(mean(df['Sales'])).collect()
mean_val

[Row(avg(Sales)=400.5)]

In [111]:
mean_sales = mean_val[0][0]
mean_sales

400.5

In [112]:
df.na.fill(mean_sales, ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [113]:
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [114]:
df3 = df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales'])

In [115]:
df3.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [116]:
df= spark.read.csv('files/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/appl_stock.csv', inferSchema=True, header=True)

In [117]:
df.head(1)

[Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [118]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [120]:
# df.select(['Date', 'Volume']).show()

In [124]:
from pyspark.sql.functions import (dayofmonth, dayofweek, dayofyear,
                                  month, year, weekofyear,hour,
                                  format_number, date_format)

In [126]:
df.select(month(df['Date'])).show()

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          2|
+-----------+
only showing top 20 rows



In [129]:
# df.select(year(df['Date'])).show()
newdf = df.withColumn('Year', year(df['Date']))

In [131]:
res = newdf.groupBy('Year').mean().select(['Year', "avg(Close)"])

In [132]:
res.show()

+----+------------------+
|Year|        avg(Close)|
+----+------------------+
|2015|120.03999980555547|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2012| 576.0497195640002|
|2016|104.60400786904763|
|2010| 259.8424600000002|
|2011|364.00432532142867|
+----+------------------+



In [134]:
res2 = res.withColumnRenamed('avg(Close)','Average Close')

In [136]:
res2.show()

+----+------------------+
|Year|     Average Close|
+----+------------------+
|2015|120.03999980555547|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2012| 576.0497195640002|
|2016|104.60400786904763|
|2010| 259.8424600000002|
|2011|364.00432532142867|
+----+------------------+



In [139]:
res2.select(['Year',format_number('Average Close',2).alias('Average')]).show()

+----+-------+
|Year|Average|
+----+-------+
|2015| 120.04|
|2013| 472.63|
|2014| 295.40|
|2012| 576.05|
|2016| 104.60|
|2010| 259.84|
|2011| 364.00|
+----+-------+



In [5]:
df = spark.read.csv('files/Python-and-Spark-for-Big-Data-master/Spark_DataFrame_Project_Exercise/walmart_stock.csv',inferSchema=True, header = True)

In [143]:
df.head(2)

[Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)]

In [144]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [145]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [146]:
df.explain()

== Physical Plan ==
FileScan csv [Date#1678,Open#1679,High#1680,Low#1681,Close#1682,Volume#1683,Adj Close#1684] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/mysparkub/jupyter/files/Python-and-Spark-for-Big-Data-master/Spark_D..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Date:string,Open:double,High:double,Low:double,Close:double,Volume:int,Adj Close:double>




In [148]:
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

In [6]:
describe = df.describe()

In [11]:
describe.select([
    'summary', format_number(describe['Open'].cast('float'),2).alias('Open'),
    format_number(describe['High'].cast('float'),2).alias('High'),
    format_number(describe['Low'].cast('float'),2).alias('Low'),
    format_number(describe['Close'].cast('float'),2).alias('Close'),
    format_number(describe['Volume'].cast('float'),2).alias('Volume'),
]).show()

+-------+--------+--------+--------+--------+-------------+
|summary|    Open|    High|     Low|   Close|       Volume|
+-------+--------+--------+--------+--------+-------------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,096.00|
+-------+--------+--------+--------+--------+-------------+



In [157]:
df.select([format_number('Open',2).alias('Open'), format_number('Close',2).alias('Close')]).show()

+-----+-----+
| Open|Close|
+-----+-----+
|59.97|60.33|
|60.21|59.71|
|59.35|59.42|
|59.42|59.00|
|59.03|59.18|
|59.43|59.04|
|59.06|59.40|
|59.79|59.50|
|59.18|59.54|
|59.87|59.85|
|59.79|60.01|
|59.93|60.61|
|60.75|61.01|
|60.81|60.91|
|60.75|61.39|
|61.18|61.47|
|61.80|60.97|
|60.86|60.71|
|60.47|61.30|
|61.53|61.36|
+-----+-----+
only showing top 20 rows



In [14]:
result = df.withColumn('HV Ratio',df['High'] / df['Volume'])
result.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|            HV Ratio|
+----------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|4.819714653321546E-6|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|6.290848613094555E-6|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|4.669412994783916E-6|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|7.367338463826307E-6|
|2012-01-09|         59.029999|   

In [181]:
df.orderBy(df.High.desc()).select('Date').first()

Row(Date='2015-01-13')

In [183]:
df.agg({'High':'max'}).show()

+---------+
|max(High)|
+---------+
|90.970001|
+---------+



In [184]:
df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [190]:
df.filter(df['Close']<60).count()

81

In [17]:
df.filter('Close<60').count()

81

In [20]:
df.filter('High > 80').count() / df.count() * 100

9.141494435612083

In [21]:
from pyspark.sql.functions import corr

In [22]:
df.select(corr('High', 'Volume')).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



In [33]:
yeardf = df.withColumn('Year', year(df['Date']))
maxdf = yeardf.groupBy('Year').max()
maxdf.select(['Year', format_number(maxdf['max(High)'],2)]).show()

+----+---------------------------+
|Year|format_number(max(High), 2)|
+----+---------------------------+
|2015|                      90.97|
|2013|                      81.37|
|2014|                      88.09|
|2012|                      77.60|
|2016|                      75.19|
+----+---------------------------+



In [38]:
monthdf = df.withColumn('month', month(df['Date']))
monthdf.groupBy('month').mean().select(['month', format_number('avg(Close)',2).alias('Close Avg')]).orderBy('month').show()


+-----+---------+
|month|Close Avg|
+-----+---------+
|    1|    71.45|
|    2|    71.31|
|    3|    71.78|
|    4|    72.97|
|    5|    72.31|
|    6|    72.50|
|    7|    74.44|
|    8|    73.03|
|    9|    72.18|
|   10|    71.58|
|   11|    72.11|
|   12|    72.85|
+-----+---------+

